<a href="https://colab.research.google.com/github/jawshan/CompNeuro_Summer2020/blob/master/Jawshan_summer_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#@title Data retrival
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [11]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [12]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]
#print(dat.keys())

In [ ]:
#@title Data Extraction


def extract_data(data, brain_area):
  row = {}
  row['mouse_name']     = data['mouse_name']
  row['num_v1_neurons'] = np.sum(data['brain_area']=='VISp')
  row['num_neurons'], row['num_trials'], row['num_time_bins'] = data['spks'].shape

  # Event times in seconds ∈ [0,2.5]  
  row['start_time']      = 0.0
  row['stim_onset_time'] = data['stim_onset']
  row['gocue_time']      = data['gocue']
  row['response_time']   = data['response_time']
  row['feedback_time']   = data['feedback_time']
  row['end_time']        = data['bin_size'] * row['num_time_bins']
  # Stimulus info : right and left stimulus contrasts ∈ [0,1] 
  row['contrast_right'] = data['contrast_right']
  row['contrast_left']  = data['contrast_left']
  # Action and feedback info   
  # response (left,no-go,right) :  ∈ {-1,0,1}
  # feedback (-ve,+ve)          :  ∈ {-1,1}  
  row['response']        = data['response']
  row['feedback']        = data['feedback_type']
  row['binary_feedback'] = np.array(data['feedback_type'] == 1).astype(int)

  # Spikes

  ## Extracting spikes from neurons corresponding to specified region
  visp_indexes = np.argwhere(data['brain_area']==brain_area)
  reduced_neuron_spikes = np.squeeze(data['spks'][visp_indexes,:,:],1)  
  ## Reshaping spikes to have trials dimension first : #trials, #neurons, #time_bins 
  row['spikes'] = reduced_neuron_spikes.transpose(1,0,2)

  ## Convert spikes to time & spike count per trial
  row['spike_times'] = np.zeros_like(row['spikes']).astype(np.float16)
  ax1, ax2, ax3      = np.shape(row['spikes'])
  row['spike_count'] = np.zeros((ax1, ax2, 1)).astype(np.float16)
  correction         = 1/1000
  for i, trial in enumerate(row['spike_times']):
    for j, neuron in enumerate(row['spike_times'][i]):
      row['spike_count'][i][j] = np.sum(row['spikes'][i][j])
      spike_time_list          = [10 * bin if spike !=0 else 0 for bin, spike in enumerate(row['spikes'][i][j])]
      row['spike_times'][i][j] = np.array(spike_time_list).astype(np.float16)

  row['spikes_reordered'] = reduced_neuron_spikes
  return(row)

extracted_data = np.array(list(map(lambda data: extract_data(data, 'VISp'), alldat)))
#print(extracted_data[0]['spike_count'][0][30])


In [ ]:
#@title Data Table
time_cols = ['start_time','stim_onset_time','gocue_time','response_time','feedback_time','end_time']
id_cols  = ['mouse_name']
dim_cols = ['num_v1_neurons','num_neurons','num_trials','num_time_bins','spike_times']

cols = id_cols + dim_cols

df = pd.DataFrame.from_records(extracted_data, columns=cols)

df_v1 = df[df['num_v1_neurons'] > 0]
df_v1